# Projeto 1 - Ciência dos Dados

Nome: Matheus Freitas Sant'Ana

Nome: Gabriel Salvator Benatar

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [147]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
from emoji import UNICODE_EMOJI

In [148]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\matfs\Desktop\Ciência dos Dados\P2\P1-Classificador-Naive-Bayes


___
**Carregando a base de dados com os tweets classificados como relevantes e não relevantes:**

In [149]:
dados = pd.read_excel('Uber.xlsx')
dados['Classe'] = dados.Classe.astype('category')

In [150]:
dados['Classe'].cat.categories = ['Irrelevante', 'Relevante']

In [151]:
dados.tail(10)

,Treinamento,Classe
490,@barb_lanzarin não!! espera que eu vou chamar ...,Irrelevante
491,"@olimpiabarbara @uber_brasil pois é, o cara só...",Relevante
492,eu gritando com o uber toda vez que alguma ami...,Irrelevante
493,@paoherco1 @esmareana @fercoboen @_juliocalder...,Irrelevante
494,peguei um uber agora e o motorista começou a t...,Relevante
495,thread de vídeos para usar no uber/99 quando e...,Irrelevante
496,rt @universitariaof: e o uber que se protegeu ...,Relevante
497,@quebrachera eu penso no meu pai q não consegu...,Relevante
498,e ainda teve cara de pau de dar cartãozinho de...,Irrelevante
499,minha mãe bebeu ousadia pra crc ontem que falo...,Irrelevante


____

In [152]:
dados.Classe.value_counts()

Irrelevante    356
Relevante      144
Name: Classe, dtype: int64

Há muito mais tweets irrelevantes do que relevantes. Vamos tentar deixar a quantidade mais ou menos parecida.

In [153]:
dados_irr = dados.loc[dados.Classe == 'Irrelevante', :]
dados_irr = dados_irr.loc[:207, :]

In [154]:
dados_irr.Classe.value_counts()

Irrelevante    144
Relevante        0
Name: Classe, dtype: int64

In [155]:
dados_r = dados.loc[dados.Classe == 'Relevante', :]

In [156]:
dados_r.Classe.value_counts()

Relevante      144
Irrelevante      0
Name: Classe, dtype: int64

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Função que limpa a pontuação e outros símbolos irrelevantes para a classificação dos _tweets_.

In [157]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@_]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

___
Limpando os _tweets_ do _DataFrame_

In [158]:
for i in range(len(dados['Treinamento'])):
    dados['Treinamento'][i] = cleanup(dados['Treinamento'][i].lower())

C:\Users\matfs\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [159]:
dados.head()

,Treinamento,Classe
0,pior coisa é ouvir de gente privilegiada que a...,Irrelevante
1,eu morrendo de calor e o uber geladinho tudo q...,Irrelevante
2,to lembrando aqui do dia em que a mairine e a ...,Irrelevante
3,lucasp711 to cortando o cabelo se ainda tive...,Irrelevante
4,rt jungsbloomie acabei o dinheiro da semana ...,Irrelevante


___
Criando os _DataFrames_ das categorias de classificação.

In [160]:
dados_irr = dados.loc[dados.Classe == 'Irrelevante', :]
dados_irr = dados_irr.loc[:209, :]

In [161]:
dados_r = dados.loc[dados.Classe == 'Relevante', :]

___
Aqui corrigimos o index de cada _DataFrame_ (eles tinham herdado o index do _DataFrame_ total).

In [162]:
for i in range(len(dados_r.index.values)):
    dados_r.index.values[i] = i 

In [163]:
for i in range(len(dados_irr.index.values)):
    dados_irr.index.values[i] = i 

___
Criando uma lista com as palavras de cada _tweet_ do _DataFrame_ dos irrelevantes.

In [164]:
todas_palavras_irrelevante = []
for i in range(len(dados_irr.Treinamento)):
    todas_palavras_irrelevante.append(dados_irr['Treinamento'][i].split())
todas_palavras_irrelevante[:2]

[['pior',
  'coisa',
  'é',
  'ouvir',
  'de',
  'gente',
  'privilegiada',
  'que',
  'anda',
  'de',
  'carro',
  'todo',
  'dia',
  'seja',
  'uber',
  'ou',
  'carro',
  'do',
  'pai',
  'que',
  'vai',
  'ter',
  'que',
  'andar',
  'de',
  'ônibus',
  'realmente',
  'essas',
  'pessoas',
  'precisam',
  'amadurecer'],
 ['eu',
  'morrendo',
  'de',
  'calor',
  'e',
  'o',
  'uber',
  'geladinho',
  'tudo',
  'q',
  'eu',
  'queria']]

Essa lista é composta de várias outras listas, em que cada uma delas corresponde às palavras de certo _tweet_. Vamos fazer com que a lista de palavras seja apenas UMA grande lista composta apenas pelas _strings_.

In [165]:
todas_palavras_irr = []
for i in range(len(todas_palavras_irrelevante)):
    for j in range(len(todas_palavras_irrelevante[i])):
        todas_palavras_irr.append(todas_palavras_irrelevante[i][j])
todas_palavras_irr[:35]

['pior',
 'coisa',
 'é',
 'ouvir',
 'de',
 'gente',
 'privilegiada',
 'que',
 'anda',
 'de',
 'carro',
 'todo',
 'dia',
 'seja',
 'uber',
 'ou',
 'carro',
 'do',
 'pai',
 'que',
 'vai',
 'ter',
 'que',
 'andar',
 'de',
 'ônibus',
 'realmente',
 'essas',
 'pessoas',
 'precisam',
 'amadurecer',
 'eu',
 'morrendo',
 'de',
 'calor']

Vamos tornar essa lista em um _DataFrame_.

In [166]:
serie_Irr = pd.Series(todas_palavras_irr)
serie_Irr.head()

0     pior
1    coisa
2        é
3    ouvir
4       de
dtype: object

Todo o processo feito acima é repetido para o _DataFrame_ dos _tweets_ relevantes.

In [167]:
todas_palavras_relevante = []
for i in range(len(dados_r.Treinamento)):
    todas_palavras_relevante.append(dados_r['Treinamento'][i].split())

In [168]:
todas_palavras_r = []
for i in range(len(todas_palavras_relevante)):
    for j in range(len(todas_palavras_relevante[i])):
        todas_palavras_r.append(todas_palavras_relevante[i][j])
todas_palavras_r[:5]

['rt', 'leandra', 'bdz', 'meninas', 'um']

In [169]:
serie_r = pd.Series(todas_palavras_r)
serie_r.head()

0         rt
1    leandra
2        bdz
3    meninas
4         um
dtype: object

___
Fazendo as tabelas com as contagens absolutas.

In [170]:
tabela_r = serie_r.value_counts()
tabela_r.head()

uber    146
de       92
e        79
o        77
que      75
dtype: int64

In [171]:
tabela_irr = serie_Irr.value_counts()
tabela_irr.head()

uber    147
o        81
de       69
e        67
eu       52
dtype: int64

___
Fazendo as tabelas com as contagens relativas.

In [172]:
tabela_relativa_r = (serie_r.value_counts(True)*100).round(2)
tabela_relativa_r.head()

uber    4.85
de      3.06
e       2.63
o       2.56
que     2.49
dtype: float64

In [173]:
tabela_relativa_irr = (serie_Irr.value_counts(True)*100).round(2)
tabela_relativa_irr.head()

uber    5.40
o       2.98
de      2.53
e       2.46
eu      1.91
dtype: float64

___
Fazendo todo o processo acima só que para o _DataFrame_ total.

In [174]:
todas_palavras_v1 = []
for i in range(len(dados.index.values)):
    todas_palavras_v1.append(dados.Treinamento[i].split())

In [175]:
todas_palavras = []
for i in range(len(todas_palavras_v1)):
    for j in range(len(todas_palavras_v1[i])):
        todas_palavras.append(todas_palavras_v1[i][j])
todas_palavras[:5]

['pior', 'coisa', 'é', 'ouvir', 'de']

In [176]:
serie_todas = pd.Series(todas_palavras)
serie_todas.head()

0     pior
1    coisa
2        é
3    ouvir
4       de
dtype: object

In [177]:
tabela_todas = serie_todas.value_counts()
tabela_todas.head()

uber    500
de      281
o       272
e       219
que     201
dtype: int64

In [178]:
tabela_relativa_todas = (serie_todas.value_counts(True)*100).round(2)
tabela_relativa_todas.head()

uber    4.88
de      2.74
o       2.65
e       2.14
que     1.96
dtype: float64

___
Função do classificador Naive Bayes

In [195]:
def naive_bayes(frase, tabela_relativa_r, tabela_relativa_irr):
    
    # Filtrando a frase tirando pontuação e símbolos irrelevantes
    frase = cleanup(frase.lower())
    
    # Separando as palavras da frase e colocando-as em uma lista.
    lista_palavras = frase.split()
    
    # Quando há o uso de emojis e eles estão 'colados', separá-los para que sejam
    # considerados palavras diferentes.
    k = 0
    for m in range(len(lista_palavras)):
        for i in lista_palavras[m]:
            if i in UNICODE_EMOJI:
                k += 1
        if k == len(lista_palavras[m]):
            sub = ",".join(lista_palavras[m])
            a = sub.split(",")
            lista_palavras.remove(lista_palavras[m])
            for s in range(len(a)):
                lista_palavras.append(a[s])
        k = 0
        
    # Contando o total de palavras em cada categoria com as contagens absolutas.
    totalR = tabela_r.value_counts().sum()
    totalIrr = tabela_irr.value_counts().sum()
    total = totalR + totalIrr
    
    # P(R)
    probR = totalR/total
    
    # P(Rc)
    probIrr=totalIrr/total
    
    # P(frase|R)
    probFraseDadoR = 100
    for i in lista_palavras:
        if i in tabela_relativa_r:
            probFraseDadoR *= tabela_relativa_r['{}'.format(i)]
        else:
            probFraseDadoR *= 0.01
    
    # P(frase|Rc)
    probFraseDadoIrr = 100
    for i in lista_palavras:
        if i in tabela_relativa_irr:
             probFraseDadoIrr *= tabela_relativa_irr['{}'.format(i)]
        else:
            probFraseDadoIrr *= 0.01
    
    # P(R|frase)
    probRDadoFrase = probFraseDadoR*probR
    
    # P(Rc|frase)
    probIrrDadoFrase = probFraseDadoIrr*probIrr
    
    if probRDadoFrase > probIrrDadoFrase:
        return 'Relevante'
    else:
        return 'Irrelevante'
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [196]:
naive_bayes('meu deus o uber que eu peguei hoje era filho de italiano 😭😷😷😷', tabela_relativa_r, tabela_relativa_irr)

'Relevante'

___
### Concluindo

In [197]:
dados_teste = pd.read_excel('Uber.xlsx', sheet_name = "Teste")

In [198]:
dados_teste['Classe'] = dados_teste.Classe.astype('category')
dados_teste['Classe'].cat.categories = ['Irrelevante', 'Relevante']

In [199]:
dados_teste['Bayes'] = 0

for i in range(len(dados_teste.index)):
    dados_teste['Bayes'][i] = naive_bayes(dados_teste['Teste'][i], tabela_relativa_r, tabela_relativa_irr)

C:\Users\matfs\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\matfs\Downloads\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [202]:
dados_teste.Bayes.value_counts()

Relevante      138
Irrelevante     62
Name: Bayes, dtype: int64

In [203]:
dados_teste.Classe.value_counts()

Irrelevante    159
Relevante       41
Name: Classe, dtype: int64

In [206]:
acertos_relevantes = 0
acertos_irrelevantes = 0
erros_relevantes = 0
erros_irrelevantes = 0

for i in range(len(dados_teste.index)):
    if dados_teste.Classe[i] == dados_teste.Bayes[i] and dados_teste.Classe[i] == 'Irrelevante':
        acertos_irrelevantes += 1
    elif dados_teste.Classe[i] == dados_teste.Bayes[i] and dados_teste.Classe[i] == 'Relevante':
        acertos_relevantes += 1
    elif dados_teste.Classe[i] != dados_teste.Bayes[i] and dados_teste.Classe[i] == 'Irrelevante':
        erros_irrelevantes += 1
    elif dados_teste.Classe[i] != dados_teste.Bayes[i] and dados_teste.Classe[i] == 'Relevante':
        erros_relevantes += 1
        
print("Acertos relevantes: {}".format(acertos_relevantes))
print("Erros relevantes: {}".format(erros_relevantes))
print("Acertos irrelevantes: {}".format(acertos_irrelevantes))
print("Erros irrelevantes: {}".format(erros_irrelevantes))

Acertos relevantes: 38
Erros relevantes: 3
Acertos irrelevantes: 59
Erros irrelevantes: 100


In [213]:
porcentagem_acerto_relevancia = acertos_relevantes/(acertos_relevantes+erros_relevantes)
porcentagem_acerto_irrelevancia = acertos_irrelevantes/(acertos_irrelevantes+erros_irrelevantes)

print("Porcentagem de acertos (relevância): {}".format((porcentagem_acerto_relevancia*100)))
print("Porcentagem de acertos (irrelevância): {}".format((porcentagem_acerto_irrelevancia*100)))

Porcentagem de acertos (relevância): 92.6829268292683
Porcentagem de acertos (irrelevância): 37.10691823899371


___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**